# Linear Regression in Keras

## 1. import libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras as K
import numpy as np
import matplotlib.pyplot as plt

## 2. Hyperparameters

In [ ]:
NUM_HIDDEN_UNITS = 64
EPOCHS = 500

## 3. Load the data

In [ ]:
# Load the Boston Housing Prices dataset
boston_housing = K.datasets.boston_housing
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

# Shuffle the training set
order = np.argsort(np.random.random(train_labels.shape))
train_data = train_data[order]
train_labels = train_labels[order]

print("Training set: {}".format(train_data.shape))  # 404 examples, 13 features
print("Testing set:  {}".format(test_data.shape))  # 102 examples, 13 features

## 4. Normalize the data

In [ ]:
# Normalize features
# Test data is *not* used when calculating the mean and std
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
train_data = (train_data - mean) / std
test_data = (test_data - mean) / std

## 5. Create the model (i.e. Graph)

In [ ]:
# Build the model
model = K.Sequential()
model.add(K.layers.Dense(NUM_HIDDEN_UNITS, activation='relu', input_shape=(train_data.shape[1],)))
model.add(K.layers.Dense(1, activation='linear'))
model.compile(loss='mse',
              optimizer=tf.train.RMSPropOptimizer(learning_rate=0.001),
              metrics=['mae'])
model.summary()


## 6. Train

In [ ]:
# Train the model
# Display training progress by printing a single dot for each completed epoch
class PrintDot(K.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 50 == 0:
            print('Epoch #{}'.format(epoch))


# The patience parameter is the amount of epochs to check for improvement
early_stop = K.callbacks.EarlyStopping(monitor='val_loss', patience=20)
history = model.fit(train_data, train_labels, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[early_stop, PrintDot()])

In [ ]:
plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Mean Abs Error [1000$]')
plt.plot(history.epoch, np.array(history.history['mean_absolute_error']), label='Train Loss')
plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']), label='Val loss')
plt.legend()
plt.ylim([0, 5])

## Test

In [ ]:
[loss, mae] = model.evaluate(test_data, test_labels, verbose=0)
print("\n Testing set Mean Abs Error: ${:7.2f}".format(mae * 1000))

In [ ]:
# Predict
test_predictions = model.predict(test_data).flatten()

plt.figure()
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [1000$]')
plt.ylabel('Predictions [1000$]')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
plt.plot([-100, 100], [-100, 100])

In [ ]:
plt.figure()
error = test_predictions - test_labels
plt.hist(error, bins=50)
plt.xlabel("Prediction Error [1000$]")
plt.ylabel("Count")
plt.show()
print()